<a href="https://colab.research.google.com/github/JonasHeymans/appartementen/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('/all.csv', index_col='Unnamed: 0')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29,37,58,92,116,120,136,139,140,141,144,146,150,152,154,162,169,170,172,173,174,175,177,180,181,182,183,184,186,187,188,189,190,191,193,194,195,196,197,198,200,201,203,205,206,207,209,210,211,212,213) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.isnull().sum(axis = 0)

cluster.bedroomRange    4024
cluster.maxPrice        4022
cluster.maxRoom         4178
cluster.maxSurface      4036
cluster.minPrice        4022
                        ... 
Extra information       4176
Reserve price           4121
Single session          4121
Terms of visit          4122
Venue of the sale       4121
Length: 213, dtype: int64

In [4]:
print(df.head().T)

                        0    1    2    3    4
cluster.bedroomRange  NaN  NaN  NaN  NaN  NaN
cluster.maxPrice      NaN  NaN  NaN  NaN  NaN
cluster.maxRoom       NaN  NaN  NaN  NaN  NaN
cluster.maxSurface    NaN  NaN  NaN  NaN  NaN
cluster.minPrice      NaN  NaN  NaN  NaN  NaN
...                   ...  ...  ...  ...  ...
Extra information     NaN  NaN  NaN  NaN  NaN
Reserve price         NaN  NaN  NaN  NaN  NaN
Single session        NaN  NaN  NaN  NaN  NaN
Terms of visit        NaN  NaN  NaN  NaN  NaN
Venue of the sale     NaN  NaN  NaN  NaN  NaN

[213 rows x 5 columns]


In [5]:
df = df.drop(columns=['customerLogoUrl','customerName','isBookmarked','media.pictures','flags.secondary'], axis =1)

In [6]:
for column in df.columns:
    if df[column].isnull().sum() > 0.8*len(df):
        df = df.drop(column, axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4178 entries, 0 to 4177
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   flags.main                                   983 non-null    object 
 1   id                                           4178 non-null   float64
 2   price.accessibilityPrice                     4178 non-null   object 
 3   price.label                                  4178 non-null   object 
 4   price.language                               4178 non-null   object 
 5   price.mainDisplayPrice                       4178 non-null   object 
 6   price.mainValue                              4019 non-null   float64
 7   price.shortDisplayPrice                      4178 non-null   object 
 8   price.type                                   4178 non-null   object 
 9   property.bedroomCount                        4019 non-null   float64
 10  

In [ ]:
df['flags.main'].unique()

In [8]:
from sklearn.impute import SimpleImputer

ids = df['id']
numerical = df.select_dtypes(['int','float'])
categorical = df.select_dtypes('object')

numerical[numerical.isnull()] = 0
# categorical[categorical.isnull()] = 'missing'

numerical_cols = numerical.columns
categorical_cols = categorical.columns

imp = SimpleImputer(strategy='mean')
numerical= imp.fit_transform(numerical)

# imp = SimpleImputer(strategy='most_frequent')
# categorical= imp.fit_transform(categorical)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [9]:
from sklearn.preprocessing import OneHotEncoder
categorical = pd.get_dummies(categorical)

In [10]:
categorical.head()

,flags.main_new,flags.main_under_option,price.accessibilityPrice_1000000€,price.accessibilityPrice_100000€,price.accessibilityPrice_100000€ + 1405€ per month,price.accessibilityPrice_1004000€,price.accessibilityPrice_101500€,price.accessibilityPrice_102279€,price.accessibilityPrice_1030000€,price.accessibilityPrice_1050000€,price.accessibilityPrice_105000€,price.accessibilityPrice_1055000€,price.accessibilityPrice_106000€,price.accessibilityPrice_107000€,price.accessibilityPrice_107500€,price.accessibilityPrice_1090000€,price.accessibilityPrice_109000€,price.accessibilityPrice_1095000€,price.accessibilityPrice_1100000€,price.accessibilityPrice_110000€,price.accessibilityPrice_1122500€,price.accessibilityPrice_1127500€,price.accessibilityPrice_1140000€,price.accessibilityPrice_114950€,price.accessibilityPrice_1150000€,price.accessibilityPrice_115000€,price.accessibilityPrice_116900€,price.accessibilityPrice_1175000€,price.accessibilityPrice_117500€,price.accessibilityPrice_1180000€,price.accessibilityPrice_118000€,price.accessibilityPrice_118450€,price.accessibilityPrice_1190000€,price.accessibilityPrice_119000€,price.accessibilityPrice_1195000€,price.accessibilityPrice_119500€,price.accessibilityPrice_1198000€,price.accessibilityPrice_1199000€,price.accessibilityPrice_1200000€,price.accessibilityPrice_120000€,...,Terrace surface_54m²square meters,Terrace surface_55m²square meters,Terrace surface_56m²square meters,Terrace surface_57m²square meters,Terrace surface_58m²square meters,Terrace surface_59m²square meters,Terrace surface_5m²square meters,Terrace surface_60m²square meters,Terrace surface_61m²square meters,Terrace surface_62m²square meters,Terrace surface_63m²square meters,Terrace surface_64m²square meters,Terrace surface_65m²square meters,Terrace surface_66m²square meters,Terrace surface_67m²square meters,Terrace surface_68m²square meters,Terrace surface_69m²square meters,Terrace surface_6m²square meters,Terrace surface_70m²square meters,Terrace surface_72m²square meters,Terrace surface_74m²square meters,Terrace surface_75m²square meters,Terrace surface_76m²square meters,Terrace surface_78m²square meters,Terrace surface_79m²square meters,Terrace surface_7m²square meters,Terrace surface_80m²square meters,Terrace surface_81m²square meters,Terrace surface_85m²square meters,Terrace surface_88m²square meters,Terrace surface_8m²square meters,Terrace surface_90m²square meters,Terrace surface_91m²square meters,Terrace surface_92m²square meters,Terrace surface_93m²square meters,Terrace surface_95m²square meters,Terrace surface_97m²square meters,Terrace surface_9m²square meters,Basement_Yes,Elevator_Yes
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
categorical['id'] = ids
df = df.drop(columns=categorical_cols)

In [12]:
numerical = pd.DataFrame(data=numerical, columns=numerical_cols)

df = numerical.merge(categorical, on='id')

In [13]:

# plt.figure(figsize=(20,20))
# chart = sns.heatmap(df.isnull(), cbar_kws={'fraction' : 0.01}, # shrink colour bar
#     cmap='OrRd')
# chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
#

In [14]:
from sklearn.model_selection import train_test_split
features =  list(df.drop(columns=['price.mainValue','Unnamed: 0.1','transaction.sale.price'],axis=1))
X = df[features]
y = df['price.mainValue']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
# from sklearn.preprocessing import StandardScaler
#
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [17]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
dummy_median = DummyRegressor(strategy='mean')
dummy_regressor = dummy_median.fit(X_train,y_train)
dummy_predicts = dummy_regressor.predict(X_test)
print("Model Accuracy:", dummy_regressor.score(X_test,y_test)*100)
print('$',mean_absolute_error(y_test,dummy_predicts))

Model Accuracy: -0.03472146060710113
$ 219247.21175835084


In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
models = [RandomForestRegressor(n_estimators=200,criterion='mse',max_depth=20,random_state=100,verbose=2),
          DecisionTreeRegressor(criterion='mse',max_depth=11,random_state=100),
          GradientBoostingRegressor(n_estimators=200,max_depth=12, verbose=2)]
learning_mods = pd.DataFrame()
temp = {}

In [ ]:
#run through models
for model in models:
    print(model)
    m = str(model)
    temp['Model'] = m[:m.index('(')]
    model.fit(X_train, y_train)
    temp['R2_Price'] = r2_score(y_test, model.predict(X_test))
    print('score on training',model.score(X_train, y_train))
    print('r2 score',r2_score(y_test, model.predict(X_test)),'\n')
    learning_mods = learning_mods.append([temp])
learning_mods.set_index('Model', inplace=True)

fig, axes = plt.subplots(ncols=1, figsize=(30, 12))
learning_mods.R2_Price.plot(ax=axes, kind='bar', title='R2_Price')
plt.show()

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=100, verbose=2, warm_start=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 200


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  4.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.2s finished


score on training 0.9635457150593564
r2 score 0.8239772748256916 

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=11,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=100, splitter='best')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


score on training 0.9478453369673673
r2 score 0.7017656757863897 

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=12,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=2, warm_start=False)
      Iter       Train Loss   Remaining Time 
         1 156571029530.4731            5.44m
         2 128179014901.3505            5.37m
         3 104914252153.8785            5.28m
         4 86050249772.4190            5.25m
         5 70763085954.6678          

In [ ]:
regressionTree_imp = model.feature_importances_
# plt.figure(figsize=(16,6))
# plt.yscale('log',nonposy='clip')
# plt.bar(range(len(regressionTree_imp)),regressionTree_imp,align='center')
# plt.xticks(range(len(regressionTree_imp)),features,rotation='vertical')
# plt.title('Feature Importance')
# plt.ylabel('Importance')
# plt.show()

In [ ]:
data_tuples = list(zip(features,regressionTree_imp.tolist()))
features_importance = pd.DataFrame(data_tuples, columns=['Feature','Value'])

In [ ]:
print(len(features_importance))
unimportant_features = features_importance[features_importance['Value'] < .001]
features_importance = features_importance[features_importance['Value'] > .001]
print(len(features_importance))

In [ ]:
features_importance = features_importance.sort_values(by=['Value'], ascending=False)
pd.set_option('display.float_format', lambda x: '%.1000f' % x)
features_importance['Value']

In [ ]:
data_tuples = list(zip(y_test,model.predict(X_test)))
features_importance = pd.DataFrame(data_tuples, columns=['Echte Waarde','Voorspelling'])
features_importance['Voorspelling'] = features_importance['Voorspelling'].astype(float)

In [ ]:
lst = unimportant_features['Feature'].tolist()
lst

In [ ]:
lst = features_importance['Feature'].tolist()
lst
